In [12]:
# importer les modules
import csv, math
import numpy as np
import pandas as pd

In [13]:
# Lecture des fichiers texte en CSV
# ouvrir et lire le fichier concerne
fichier ="liste2.csv"
fichier_lire=open(fichier,"r")
fichier_csv = csv.reader(fichier_lire,delimiter=",")
tab=list(fichier_csv) 
mvector=np.asarray(tab)
x1=mvector[1:,0]
#print(x1)
print(mvector[1:,:])

[['1' '1754' '23' ..., '0' '20' '0']
 ['2' '1755' '21' ..., '0' '20' '0']
 ['3' '43493' '25' ..., '1' '30' '0']
 ..., 
 ['309' '43625' '23' ..., '1' '30' '0']
 ['310' '1048' '25' ..., '0' '20' '0']
 ['311' '1050' '79' ..., '0' '20' '0']]


In [23]:
fichier_lire=open(fichier,"r")
csv = pd.read_csv(fichier_lire,delimiter=",")
x_mean=csv.mean()

mean=np.asarray(x_mean)
print(mean)
cov=np.cov(mvector[1:,:].astype(np.float),rowvar=0)

print(cov.shape)
print(mvector[1:,:].shape)


[  1.57627119e+02   8.88547797e+03   3.54021695e+03   1.92000000e+02
   1.68000000e+02   7.45762712e-01   3.09152542e+01   1.92000000e+02
   1.68000000e+02   2.54237288e-01   3.36610169e+01   1.32203390e-01]
(12, 12)
(295, 12)
[[  8.15810538e+03   8.65260258e+04  -1.39175667e+05   0.00000000e+00
    0.00000000e+00   2.02052346e+00   5.66689727e+01   0.00000000e+00
    0.00000000e+00  -2.02052346e+00   1.40228295e+01   1.04372190e+01]
 [  8.65260258e+04   2.56190197e+08  -3.07824121e+07   0.00000000e+00
    0.00000000e+00  -5.40940868e+03   6.62554794e+04   0.00000000e+00
    0.00000000e+00   5.40940868e+03  -2.71977082e+04  -9.32162043e+02]
 [ -1.39175667e+05  -3.07824121e+07   1.29967780e+08   0.00000000e+00
    0.00000000e+00   7.48279834e+02  -4.58434890e+03   0.00000000e+00
    0.00000000e+00  -7.48279834e+02   1.98277745e+04  -3.67409731e+02]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
